In [1]:
from datetime import datetime
import numpy as np
import bw2data as bd
from bw_temporalis import TemporalDistribution
from optimex import lca_processor

# Set the project (creates it if not existing)
bd.projects.set_current("__test_standalone_db__")

# BIOSPHERE
biosphere_data = {
    ("biosphere3", "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "CAS number": "000124-38-9"
    },
    ("biosphere3", "CH4"): {
        "type": "emission",
        "name": "methane, fossil",
        "CAS number": "000074-82-8"
    },
}
bd.Database("biosphere3").write(biosphere_data)

# BACKGROUND 2020
db_2020_data = {
    ("db_2020", "I1"): {
        "name": "node I1",
        "location": "somewhere",
        "reference product": "I1",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2020", "I1")},
            {"amount": 1, "type": "biosphere", "input": ("biosphere3", "CO2")},
        ],
    },
    ("db_2020", "I2"): {
        "name": "node I2",
        "location": "somewhere",
        "reference product": "I2",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2020", "I2")},
            {"amount": 1, "type": "biosphere", "input": ("biosphere3", "CH4")},
        ],
    },
}
bd.Database("db_2020").write(db_2020_data)

# BACKGROUND 2030
db_2030_data = {
    ("db_2030", "I1"): {
        "name": "node I1",
        "location": "somewhere",
        "reference product": "I1",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2030", "I1")},
            {"amount": 0.9, "type": "biosphere", "input": ("biosphere3", "CO2")},
        ],
    },
    ("db_2030", "I2"): {
        "name": "node I2",
        "location": "somewhere",
        "reference product": "I2",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2030", "I2")},
            {"amount": 0.9, "type": "biosphere", "input": ("biosphere3", "CH4")},
        ],
    },
}
bd.Database("db_2030").write(db_2030_data)

# FOREGROUND - temporally distributed
foreground_data = {
    ("foreground", "P1"): {
        "name": "process P1",
        "location": "somewhere",
        "reference product": "F1",
        "exchanges": [
            {
                "amount": 1,
                "type": "production",
                "input": ("foreground", "P1"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
                "operation": True,
            },
            {
                "amount": 27.5,
                "type": "technosphere",
                "input": ("db_2020", "I1"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([1, 0, 0, 0]),
                ),
            },
            {
                "amount": 20,
                "type": "biosphere",
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
                "operation": True,
            },
        ],
    },
    ("foreground", "P2"): {
        "name": "process P2",
        "location": "somewhere",
        "reference product": "F1",
        "exchanges": [
            {
                "amount": 1,
                "type": "production",
                "input": ("foreground", "P2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
                "operation": True,
            },
            {
                "amount": 1,
                "type": "technosphere",
                "input": ("db_2020", "I2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([1, 0, 0, 0]),
                ),
            },
            {
                "amount": 20,
                "type": "biosphere",
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
                "operation": True,
            },
        ],
    },
}
bd.Database("foreground").write(foreground_data)


bd.Method(("GWP", "example")).write([
    (("biosphere3", "CO2"), 1),
    (("biosphere3", "CH4"), 27),
])

bd.Method(("land use", "example")).write([
    (("biosphere3", "CO2"), 2),
    (("biosphere3", "CH4"), 1),
])

100%|██████████| 2/2 [00:00<?, ?it/s]


16:11:35+0200 [info     ] Vacuuming database            
16:11:35+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


16:11:35+0200 [info     ] Vacuuming database            
16:11:35+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


16:11:35+0200 [info     ] Vacuuming database            
16:11:35+0200 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


16:11:35+0200 [info     ] Vacuuming database            


In [2]:
from optimex import lca_processor


# For optimex to work, we need to manually define which processes  produce desired 
# products labeled as functional flows
foreground = bd.Database("foreground")
for act in foreground:
    act["functional flow"] = "F1"
    act.save()

# Define temporally distirbuted demand from 2020 to 2030
td_demand = TemporalDistribution(
    date=np.arange(2020 - 1970, 10, dtype="datetime64[Y]"),
    amount=np.asarray([0, 0, 10, 5, 10, 5, 10, 5, 10, 5]),
)

lca_config = lca_processor.LCAConfig(
    demand = {"F1": td_demand},
    temporal= {
        "start_date": datetime(2020, 1, 1),
        "temporal_resolution": "year",
        "timehorizon": 100,
        "database_dates": {
            "db_2020": datetime(2020, 1, 1),
            "db_2030": datetime(2030, 1, 1),
            "foreground": "dynamic",
        },
    },
    characterization_methods=[
        {
            "category_name": "climate_change",
            "brightway_method": ("GWP", "example"),
            "metric": "CRF",
        },
        {
            "category_name": "land_use",
            "brightway_method": ("land use", "example"),
        },
    ],
    background_inventory={
        "cutoff": 1e4,
        "calculation_method": "sequential",
    },
)

In [3]:
lca_data_processor = lca_processor.LCADataProcessor(lca_config)

2025-05-22 16:11:35.898 | INFO     | optimex.lca_processor:_parse_demand:317 - Identified demand in system time range of %s for functional flows %s
2025-05-22 16:11:35.906 | INFO     | optimex.lca_processor:_construct_foreground_tensors:437 - Constructed foreground tensors.
2025-05-22 16:11:35.910 | INFO     | optimex.lca_processor:log_tensor_dimensions:432 - Technosphere shape: (2 processes, 2 flows, 4 years) with 8 total entries.
2025-05-22 16:11:35.911 | INFO     | optimex.lca_processor:log_tensor_dimensions:432 - Biosphere shape: (2 processes, 1 flows, 4 years) with 8 total entries.
2025-05-22 16:11:35.913 | INFO     | optimex.lca_processor:log_tensor_dimensions:432 - Production shape: (2 processes, 1 flows, 4 years) with 8 total entries.
2025-05-22 16:11:35.916 | INFO     | optimex.lca_processor:_calculate_inventory_of_db:475 - Calculating inventory for database: db_2020
2025-05-22 16:11:35.998 | INFO     | optimex.lca_processor:_calculate_inventory_of_db:491 - Factorized LCI for 

In [4]:
from optimex import converter

manager = converter.ModelInputManager()
model_inputs = manager.parse_from_lca_processor(lca_data_processor)

In [5]:
manager.model_inputs.model_dump()

{'PROCESS': ['P2', 'P1'],
 'FUNCTIONAL_FLOW': ['F1'],
 'INTERMEDIATE_FLOW': ['I2', 'I1'],
 'ELEMENTARY_FLOW': ['CO2', 'CH4'],
 'BACKGROUND_ID': ['db_2020', 'db_2030'],
 'PROCESS_TIME': [0, 1, 2, 3],
 'SYSTEM_TIME': [2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029],
 'CATEGORY': ['climate_change', 'land_use'],
 'demand': {('F1', 2020): 0.0,
  ('F1', 2021): 0.0,
  ('F1', 2022): 10.0,
  ('F1', 2023): 5.0,
  ('F1', 2024): 10.0,
  ('F1', 2025): 5.0,
  ('F1', 2026): 10.0,
  ('F1', 2027): 5.0,
  ('F1', 2028): 10.0,
  ('F1', 2029): 5.0},
 'operation_flows': {('P2', 'F1'): True,
  ('P2', 'CO2'): True,
  ('P1', 'F1'): True,
  ('P1', 'CO2'): True},
 'foreground_technosphere': {('P2', 'I2', 0): 1.0,
  ('P2', 'I2', 1): 0.0,
  ('P2', 'I2', 2): 0.0,
  ('P2', 'I2', 3): 0.0,
  ('P1', 'I1', 0): 27.5,
  ('P1', 'I1', 1): 0.0,
  ('P1', 'I1', 2): 0.0,
  ('P1', 'I1', 3): 0.0},
 'foreground_biosphere': {('P2', 'CO2', 0): 0.0,
  ('P2', 'CO2', 1): 10.0,
  ('P2', 'CO2', 2): 10.0,
  ('P2', 'CO2', 3): 0